In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

### inlezen van data

In [ ]:
# eerst laden we de dataset in 
df = pd.read_csv("../3dprinter/data.csv", sep = ';')
X = df.drop('tension_strength', axis = 'columns')
y = df.tension_strength
X.head()

### snel een regressie model maken

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# functie om de R2 adjusted te berekenen
def R2adjusted(R2, n, p):
    return 1-(1-R2)*(n-1)/(n-p-1)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=2)
# een lineaire regressie model maken
linreg = LinearRegression()
linreg.fit(x_train, y_train)

y_pred = linreg.predict(x_test)

In [ ]:
print('MSE: ', mean_squared_error(y_test, y_pred))
R2 = r2_score(y_test, y_pred)
n = len(y_test)
p = len(x_test.columns)
R2adj = R2adjusted(R2, n, p)
print('R2: ', R2)
print('R2adj', R2adj)

Zoals we zien is de R2 erg slecht. We kunnen de correlatiecoefficienten berekenen om te kijken of we last hebben van multicollineariteit.

In [ ]:
# maakt de correlatie matrix
X.corr()

Hoge correlaties die we kunnen vinden zijn bed temperature met fan speed, tension strength met elongation, roughness met layer height. We kunnen even wat eruit filteren door te kijken welke (absolute) correlaties hoger zijn dan 0.4.

In [ ]:
X.corr()[abs(X.corr())>0.4]

We kunnen de vif test doen om te kijken welke correlaties het hoogst zijn.

In [ ]:
# we kunnen de join functie gebruiken om een list van strings aan elkaar te verbinden met ' + ' als seperator
print(X.columns)
' + '.join(list(X.columns))

### Uitvoeren van VIF test

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from patsy import dmatrices 

# het eerste argument in dmatrices is 'y ~ X1 + X2 + ... '. Hier moet je y en X vervangen door de kolomnamen van je dataframe
y_vif, x_vif = dmatrices('tension_strength ~ ' + ' + '.join(list(X.columns)), data=df, return_type='dataframe') 

vif = pd.DataFrame() 
vif['variable'] = x_vif.columns 
vif['VIF'] = [variance_inflation_factor(x_vif.values, i) for i in range(x_vif.shape[1])]
vif

We zien hoge vif waardes. Dat betekent dat we wat kolommen moeten verwijderen. Probeer een aantal kolommen te verwijderen zodat alle vif waardes lager dan 5 zijn.

### iteratief variabelen verwijderen voor betere R2adjusted

In [ ]:
X = df.drop(['bed_temperature'], axis = 'columns')

In [ ]:
# het eerste argument in dmatrices is 'y ~ X1 + X2 + ... '. Hier moet je y en X vervangen door de kolomnamen van je dataframe
y_vif, x_vif = dmatrices('tension_strength ~ ' + ' + '.join(list(X.columns)), data=df, return_type='dataframe') 

vif = pd.DataFrame() 
vif['variable'] = x_vif.columns 
vif['VIF'] = [variance_inflation_factor(x_vif.values, i) for i in range(x_vif.shape[1])]
vif

Sommige waarden zijn nog steeds erg hoog.

In [ ]:
X = df.drop(['bed_temperature', 'nozzle_temperature'], axis = 'columns')

Herhaal deze stappen totdat je tevreden bent over de correlaties.

---

In [ ]:
# laten we dan de data standardiseren - zodat we de impact van de variabelen kunnen vergelijken
Z = (X-X.mean())/X.std()

In [ ]:
# we can make a new train/test split
x_train, x_test, y_train, y_test = train_test_split(Z, y, test_size=0.5, random_state=2)

In [ ]:
linreg = LinearRegression()
linreg.fit(x_train, y_train)
y_pred = linreg.predict(x_test)

In [ ]:
print('MSE: ', mean_squared_error(y_test, y_pred))
R2 = r2_score(y_test, y_pred)
n = len(y_test)
p = len(x_test.columns)
R2adj = R2adjusted(R2, n, p)
print('R2: ', R2)
print('R2adj', R2adj)

In [ ]:
plt.scatter(y_test, y_pred)
plt.plot([5, 35], [5, 35], 'r')

plt.xlabel('Labels')
plt.ylabel('Predictions')

plt.show()

In [ ]:
plt.barh(X.columns, linreg.coef_)

We can see that some variables do not add a lot of 'predictiveness'. We can remove those columns iteratively.

In [ ]:
# preprocess data
X = df.drop(['nozzle_temperature', 'bed_temperature'], axis = 'columns')  # add your columns here
Z = (X-X.mean())/X.std()
x_train, x_test, y_train, y_test = train_test_split(Z, y, test_size=0.5, random_state=2)

# make a linear regression model
linreg = LinearRegression()
linreg.fit(x_train, y_train)
y_pred = linreg.predict(x_test)

# calculate metrics
R2 = r2_score(y_test, y_pred)
n = len(y_test)
p = len(x_test.columns)
R2adj = R2adjusted(R2, n, p)
print('R2: ', R2)
print('R2adj', R2adj)

If done well, we see a small increase in the R2adj. That is very good. How about the beta's?

Can you try to make this model better by removing more columns?